<a href="https://colab.research.google.com/github/A-H-Sumon/Deep-Learning/blob/main/McNemarTest_PrimaryDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from joblib import Parallel, delayed
import multiprocessing

import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.feature import hog
from skimage.filters import gabor
from joblib import Parallel, delayed
import multiprocessing
from termcolor import colored

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("/content/drive/My Drive/dataset.csv", dtype=str)  # Ensure paths are read as strings

# Define features and labels
X = df["image_path"]  # Image paths
y = df["label"].astype(int)  # Labels as integers

# Split into 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y, shuffle=True)

# Create DataFrames
train_df = pd.DataFrame({"image_path": X_train, "label": y_train})
test_df = pd.read_csv('/content/drive/MyDrive/test_df.csv')




In [4]:
print(df['label'].unique())  # Check unique labels in the dataset
print(df['label'].nunique())  # Check the number of unique classes


['0' '1' '2' '3' '4' '5' '6' '7' '8' '9']
10


In [5]:
print(f"Total training images: {len(train_df)}")
print(f"Total testing images: {len(test_df)}")

Total training images: 1672
Total testing images: 418


In [6]:
import cv2
import numpy as np
import cv2.ximgproc as xip

def preprocess_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return np.zeros((28, 28))  # Placeholder for missing images

    # Apply Otsu's Thresholding
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Resize image to 28x28
    img = cv2.resize(img, (28, 28), interpolation=cv2.INTER_AREA) / 255.0

    return img


In [7]:
# ======================== Load & Preprocess Images ========================
train_images = np.array([preprocess_image(fp) for fp in train_df['image_path'].values])

In [8]:
import cv2
import numpy as np
import multiprocessing
from joblib import Parallel, delayed
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random



def augment_image(img):
    rows, cols = img.shape



    # Random Rotation
    angle = random.uniform(-15, 15)
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated = cv2.warpAffine(img, M, (cols, rows))

    # Random Shifting
    tx = random.uniform(-2, 2)
    ty = random.uniform(-2, 2)
    M_shift = np.float32([[1, 0, tx], [0, 1, ty]])
    shifted = cv2.warpAffine(rotated, M_shift, (cols, rows))

    return shifted

def augment_dataset(train_images):
    """Applies augmentation to a dataset using parallel processing."""
    num_cores = multiprocessing.cpu_count()
    augmented_images = Parallel(n_jobs=num_cores)(
        delayed(augment_image)(img) for img in train_images
    )
    return np.array(augmented_images)

# Apply augmentation to dataset
augmented_images = augment_dataset(train_images)



In [9]:
test_df['label'] = test_df['label'].astype(str)

test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',  # Include labels here
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',  # Ensure class mode is categorical
    shuffle=False
)


Found 418 validated image filenames belonging to 10 classes.


In [10]:
cores = multiprocessing.cpu_count()
print(cores)

2


In [11]:


!pip install mahotas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 54.3 MB/s eta 0:00:00


In [12]:
from joblib import Parallel, delayed
import numpy as np
import multiprocessing
from skimage.feature import hog, local_binary_pattern
from skimage.filters import gabor
from mahotas.features import zernike_moments

# Feature Extraction
def extract_features_single(img):
     # Extract HOG features
    hog_features = hog(img, pixels_per_cell=(4, 4), cells_per_block=(2, 2), feature_vector=True)

    # Extract Gabor features
    gabor_response_real, gabor_response_imag = gabor(img, frequency=0.6)
    gabor_features = np.mean(gabor_response_real) + np.mean(gabor_response_imag)

    # Extract LBP features
    lbp = local_binary_pattern(img, P=8, R=1, method="uniform")
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 10))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= lbp_hist.sum()  # Normalize LBP histogram

    return np.hstack([hog_features, gabor_features, lbp_hist])

def extract_features(images):
    num_cores = multiprocessing.cpu_count()
    features = Parallel(n_jobs=num_cores)(delayed(extract_features_single)(img) for img in images)
    return np.array(features)

# Apply feature extraction
augmented_features = extract_features(augmented_images)



In [13]:
# ======================== Data Preparation ========================
X_train_preprocessed = np.array([preprocess_image(fp) for fp in train_df['image_path']])
X_test_preprocessed = np.array([preprocess_image(fp) for fp in test_df['image_path']])

In [14]:
# Apply augmentation
X_train_augmented = augment_dataset(X_train_preprocessed)




In [15]:
# Extract features from augmented images
X_train_features = extract_features(X_train_augmented)
X_test_features = extract_features(X_test_preprocessed)  # No augmentation for test set

In [16]:
scaler = StandardScaler()
X_train_features = scaler.fit_transform(X_train_features)
X_test_features = scaler.transform(X_test_features)

encoder = LabelEncoder()
y_train = encoder.fit_transform(train_df['label'])
y_test = encoder.fit_transform(test_df['label'])

In [17]:
!pip install optuna



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 16.4 MB/s eta 0:00:00


In [18]:
import cv2
import numpy as np
from glob import glob

def load_images(image_paths):
    data = []
    for path in image_paths:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        img = cv2.resize(img, (64, 64))  # Resize to a fixed size
        img = img.flatten()  # Convert to 1D array
        data.append(img)
    return np.array(data)

# Assuming X_train originally contains image file paths
X_train = load_images(X_train)


In [19]:
import optuna
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):
    """Objective function for optimizing KNN hyperparameters."""

    # Hyperparameters to optimize
    k = trial.suggest_int('n_neighbors', 3, 15)  # Number of neighbors
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])

    # Initialize KNN classifier
    knn_model = KNeighborsClassifier(n_neighbors=k, metric=metric, weights=weights)

    # Perform cross-validation to get accuracy
    score = cross_val_score(knn_model, X_train, y_train, cv=5, scoring='accuracy').mean()

    return score  # Return the mean cross-validation accuracy

In [20]:
study_knn = optuna.create_study(direction='maximize')  # Maximize accuracy
study_knn.optimize(objective, n_trials=50)  # Run 50 trials

# Get the best parameters
best_knn_params = study_knn.best_params
print("Best KNN hyperparameters:", best_knn_params)

[I 2025-07-19 10:42:44,823] A new study created in memory with name: no-name-18540066-b805-4537-b3bd-5033c3f2ef66
[I 2025-07-19 10:42:45,785] Trial 0 finished with value: 0.7846813835016535 and parameters: {'n_neighbors': 14, 'metric': 'minkowski', 'weights': 'distance'}. Best is trial 0 with value: 0.7846813835016535.
[I 2025-07-19 10:42:54,096] Trial 1 finished with value: 0.7619715792296005 and parameters: {'n_neighbors': 6, 'metric': 'manhattan', 'weights': 'uniform'}. Best is trial 0 with value: 0.7846813835016535.
[I 2025-07-19 10:43:01,794] Trial 2 finished with value: 0.7721297703101261 and parameters: {'n_neighbors': 12, 'metric': 'manhattan', 'weights': 'distance'}. Best is trial 0 with value: 0.7846813835016535.
[I 2025-07-19 10:43:09,832] Trial 3 finished with value: 0.7589811421932255 and parameters: {'n_neighbors': 9, 'metric': 'manhattan', 'weights': 'uniform'}. Best is trial 0 with value: 0.7846813835016535.
[I 2025-07-19 10:43:10,503] Trial 4 finished with value: 0.770

Best KNN hyperparameters: {'n_neighbors': 3, 'metric': 'minkowski', 'weights': 'distance'}


In [21]:
# Extract best values
best_k = best_knn_params['n_neighbors']
best_metric = best_knn_params['metric']
best_weights = best_knn_params['weights']

def knn(X_train, y_train, X_test, k=best_k, metric=best_metric, weights=best_weights):
    """Optimized KNN model."""

    # Convert labels to NumPy array (avoid indexing issues)
    y_train = np.array(y_train)

    # Fit KNN classifier with optimized parameters
    knn_model = KNeighborsClassifier(n_neighbors=k, metric=metric, weights=weights)
    knn_model.fit(X_train, y_train)

    # Predict on test set
    predictions = knn_model.predict(X_test)

    return predictions


In [22]:


# Run KNN with optimized parameters
knn_preds = knn(X_train_features, y_train, X_test_features)


In [23]:
X_train_combined = np.hstack((X_train_features, y_train.reshape(-1, 1)))
X_test_combined = np.hstack((X_test_features, knn_preds.reshape(-1, 1)))

In [24]:
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

feature_dim = X_train_combined.shape[1]  # Assuming X_train is your feature matrix


def objective(trial):
    """Objective function for optimizing neural network hyperparameters."""

    # Hyperparameters to optimize
    num_units_1 = trial.suggest_int('num_units_1', 128, 512, step=64)  # First Dense layer
    num_units_2 = trial.suggest_int('num_units_2', 64, 256, step=64)   # Second Dense layer
    num_units_3 = trial.suggest_int('num_units_3', 32, 128, step=32)   # Third Dense layer
    dropout_1 = trial.suggest_float('dropout_1', 0.2, 0.5)  # Dropout after first layer
    dropout_2 = trial.suggest_float('dropout_2', 0.2, 0.5)  # Dropout after second layer
    learning_rate = trial.suggest_loguniform('learning_rate', 0.0001, 0.001)  # Adam learning rate

    # Build model
    model = Sequential([
        Dense(num_units_1, activation='selu', input_shape=(feature_dim,)),
        BatchNormalization(),
        Dropout(dropout_1),
        Dense(num_units_2, activation='selu'),
        BatchNormalization(),
        Dropout(dropout_2),
        Dense(num_units_3, activation='selu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])

    # Compile model with tuned learning rate
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train model (Use fewer epochs for tuning)
    history = model.fit(
        X_train_combined, y_train,
        epochs=10,  # Keep epochs low for tuning
        batch_size=64,
        validation_split=0.2,
        verbose=0
    )

    # Return best validation accuracy
    return max(history.history['val_accuracy'])


In [25]:
study = optuna.create_study(direction='maximize')  # Maximize accuracy
study.optimize(objective, n_trials=20)  # Run 20 trials

# Get the best parameters
best_params = study.best_params
print("Best hyperparameters:", best_params)

[I 2025-07-19 10:44:19,017] A new study created in memory with name: no-name-4bd8f9fa-e602-4e80-90f8-6bd036af47c2
/tmp/ipython-input-24-3243621785.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.0001, 0.001)  # Adam learning rate
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[I 2025-07-19 10:44:24,844] Trial 0 finished with value: 0.8507462739944458 and parameters: {'num_units_1': 512, 'num_units_2': 192, 'num_units_3': 64, 'dropout_1': 0.3001864029637515, 'dropout_2': 0.4762140236359

Best hyperparameters: {'num_units_1': 384, 'num_units_2': 128, 'num_units_3': 64, 'dropout_1': 0.3182822701427349, 'dropout_2': 0.39244297429839403, 'learning_rate': 0.0008050689691449271}


In [27]:
# Unpack Optuna best parameters
num_units_1 = best_params['num_units_1']
num_units_2 = best_params['num_units_2']
num_units_3 = best_params['num_units_3']
dropout_1 = best_params['dropout_1']
dropout_2 = best_params['dropout_2']
learning_rate = best_params['learning_rate']


import numpy as np
from sklearn.metrics import accuracy_score
from statsmodels.stats.contingency_tables import mcnemar

# ====================== EfficientNet-B0 ============================
# Load predictions if previously saved
efficientnet_preds = np.load("/content/drive/My Drive/test_predictions.npy")

# ====================== FusionNet Predictions ======================
# Rebuild final FusionNet model using best hyperparameters
final_model = Sequential([
    Dense(num_units_1, activation='selu', input_shape=(X_train_combined.shape[1],)),
    BatchNormalization(),
    Dropout(dropout_1),
    Dense(num_units_2, activation='selu'),
    BatchNormalization(),
    Dropout(dropout_2),
    Dense(num_units_3, activation='selu'),
    BatchNormalization(),
    Dense(10, activation='softmax')
])

final_model.compile(optimizer=Adam(learning_rate=learning_rate),
                    loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train on full training data
final_model.fit(X_train_combined, y_train, epochs=20, batch_size=64, verbose=0)

# Predict on test set
fusionnet_preds_proba = final_model.predict(X_test_combined)
fusionnet_preds = np.argmax(fusionnet_preds_proba, axis=1)



test_df['label'] = test_df['label'].astype(str)

true_labels = test_generator.labels  # Already aligned
# ====================== Contingency Table ======================
fusionnet_correct = fusionnet_preds == true_labels
efficientnet_correct = efficientnet_preds == true_labels

B = np.sum((fusionnet_correct == True) & (efficientnet_correct == False))  # FusionNet correct, EfficientNet wrong
C = np.sum((fusionnet_correct == False) & (efficientnet_correct == True))  # EfficientNet correct, FusionNet wrong

# Create table
table = [[0, B],
         [C, 0]]

# Run McNemar's Test
result = mcnemar(table, exact=False, correction=True)

# ====================== Output ======================
print("\n========= McNemar’s Test Result =========")
print(f"Contingency Table: [[0, {B}], [{C}, 0]]")
print(f"McNemar's Test Statistic: {result.statistic:.4f}")
print(f"P-value:  {result.pvalue:.4f}")

if result.pvalue > 0.05:
    print(" No statistically significant difference between FusionNet and EfficientNet-B0.")
else:
    print(" Statistically significant difference between the two models.")



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

========= McNemar’s Test Result =========
Contingency Table: [[0, 39], [19, 0]]
McNemar's Test Statistic: 6.2241
P-value:  0.0126
 Statistically significant difference between the two models.
